In [1]:
!pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
!pip install -U -q "google-genai==1.7.0"
from google import genai
from google.genai import types

genai.__version__
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.9 MB/s eta 0:00:00


**Storing Information in a DB:**

In [2]:
%load_ext sql
%sql sqlite:///gym.db

In [3]:
%%sql
CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name VARCHAR(255) NOT NULL,
    last_name VARCHAR(255) NOT NULL,
    goal VARCHAR(255) NOT NULL
);

CREATE TABLE IF NOT EXISTS user_preferences (
    preference_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    unit_system VARCHAR(50) NOT NULL, -- e.g., Metric, Imperial
    reminder_time INTEGER, -- in seconds before workout
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);

CREATE TABLE IF NOT EXISTS workouts (
    workout_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    note VARCHAR(255) NOT NULL,
    date TEXT NOT NULL, -- ISO 8601 format (YYYY-MM-DD)
    duration INTEGER, -- in minutes
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);

CREATE TABLE IF NOT EXISTS exercises_lookup (
    exercise_id INTEGER PRIMARY KEY AUTOINCREMENT,
    exercise_name VARCHAR(255) NOT NULL UNIQUE,
    muscle_group VARCHAR(255) NOT NULL
);

CREATE TABLE IF NOT EXISTS exercises (
    log_id INTEGER PRIMARY KEY AUTOINCREMENT,
    workout_id INTEGER NOT NULL,
    exercise_id INTEGER NOT NULL,
    sets INTEGER NOT NULL,
    weight INTEGER NOT NULL,
    reps INTEGER NOT NULL,
    FOREIGN KEY (workout_id) REFERENCES workouts(workout_id),
    FOREIGN KEY (exercise_id) REFERENCES exercises_lookup(exercise_id)
);

CREATE TABLE IF NOT EXISTS progress (
    progress_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    exercise_id INTEGER NOT NULL,
    date TEXT NOT NULL, -- ISO 8601 format (YYYY-MM-DD)
    weight_lifted INTEGER NOT NULL,
    reps_completed INTEGER NOT NULL,
    sets_completed INTEGER NOT NULL,
    FOREIGN KEY (user_id) REFERENCES users(user_id),
    FOREIGN KEY (exercise_id) REFERENCES exercises_lookup(exercise_id)
);

CREATE TABLE IF NOT EXISTS goals (
    goal_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    goal_description TEXT NOT NULL,
    target_date TEXT NOT NULL, -- ISO 8601 format (YYYY-MM-DD)
    achieved INTEGER DEFAULT 0, -- 0 = False, 1 = True
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);

CREATE TABLE IF NOT EXISTS routines (
    routine_id INTEGER PRIMARY KEY AUTOINCREMENT,
    routine_name VARCHAR(255) NOT NULL,
    description TEXT NOT NULL,
    difficulty_level VARCHAR(50) NOT NULL -- e.g., Beginner, Intermediate, Advanced
);

CREATE TABLE IF NOT EXISTS routine_exercises (
    routine_exercise_id INTEGER PRIMARY KEY AUTOINCREMENT,
    routine_id INTEGER NOT NULL,
    exercise_id INTEGER NOT NULL,
    sets INTEGER NOT NULL,
    reps INTEGER NOT NULL,
    rest_time_seconds INTEGER NOT NULL, -- in seconds
    FOREIGN KEY (routine_id) REFERENCES routines(routine_id),
    FOREIGN KEY (exercise_id) REFERENCES exercises_lookup(exercise_id)
);

CREATE TABLE IF NOT EXISTS user_routines (
    user_routine_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    routine_id INTEGER NOT NULL,
    start_date TEXT NOT NULL, -- ISO 8601 format (YYYY-MM-DD)
    end_date TEXT, -- ISO 8601 format (YYYY-MM-DD)
    FOREIGN KEY (user_id) REFERENCES users(user_id),
    FOREIGN KEY (routine_id) REFERENCES routines(routine_id)
);

 * sqlite:///gym.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

**Answering Basic Gym Questions:**

In [4]:
introduction = """
🤖 Welcome to Your Personal Gym Helper! 💪

I’m here to guide you safely and effectively on your fitness journey. Whether you’re just starting out or looking to refine your routine, I’ll provide clear, actionable advice tailored to your goals.

Safety is my top priority—always listen to your body and consult a professional if needed.

Let’s get started! 😊

What’s your primary fitness goal? (e.g., lose weight, build muscle, improve endurance)
"""

In [5]:
import sqlite3
db_file = "gym.db"
db_conn = sqlite3.connect(db_file)

def list_tables() -> list[str]:
    print(' DB CALL:list_tables()')
    cursor=db_conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables=cursor.fetchall()
    return [t[0] for t in tables]

def describe_table(table_name:str) -> list[tuple[str,str]]:
    print(f'-DB CALL: describe_table({table_name})')
    cursor=db_conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name});")
    schema=cursor.fetchall()
    return [(col[1],col[2]) for col in schema]

def execute_query(sql:str) -> list[list[str]]:
    print(f' -DB CALL: execute_query({sql})')
    cursor=db_conn.cursor()
    cursor.execute(sql)
    return cursor.fetchall()

In [6]:
db_tools = [list_tables, describe_table, execute_query]

instruction = """You are a helpful chatbot that can interact with an SQL database
as a personal gym helper. You will take the users questions and turn them into SQL
queries using the tools available. Once you have the information you need, you will
answer the user's question using the data returned.

Use list_tables to see what tables are present, describe_table to understand the
schema, and execute_query to issue an SQL SELECT query."""

client = genai.Client(api_key=GOOGLE_API_KEY)

chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=instruction,
        tools=db_tools,
    ),
)